In [ ]:
"""
MLB Contract Regression v2 - Strategic Feature Engineering

Intended to incorporate new feature engineering

New features include:
    ISI * Age/Performance interactions
    Pitcher-specific injury features
    More detailed injury type breakdowon
    Injury temporal trends, analyzing positive or negative injury history trends
    Position-specific encoding

    """


In [ ]:
# package imports
from __future__ import annotations

import time
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple
from itertools import product

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from xgboost import XGBRegressor


In [ ]:
# Configuration

DATA_PATH = r"contracts_with_isi_v2_SWEEP_WIDE_WITH_KEYS_PLUS_CPI.csv"
BAT_RATES_PATH = r"batting_rates_by_season.csv"
PIT_RATES_PATH = r"pitching_rates_by_season.csv"
DEF_STATS_PATH = r"defensive_stats.csv"
STATCAST_PIT_PATH = r"statcast_pitching_2015_2025.csv"


# Output path for results and model weights
OUT_RESULTS = r"regression_v2_results.csv"

# establishes time limit for how long the model can run
TIME_LIMIT_HOURS = 2
RANDOM_STATE = 42

# Time split
TRAIN_YEARS = [2020, 2021, 2022, 2023]
TEST_YEARS = [2024, 2025]

# Contract filters
# utilzied in config rather than further down, as in original baseline model
# max years filtered max contract term
# top pctl filters top_n percent of contract AAV
MAX_YEARS = 5
REMOVE_TOP_PCTL = 0.95

# Target
REG_TARGET = "guarantee_real_per_year_2025"

Total parameter combinations: 614,400


In [ ]:
# Hyperparameter Grid Search
# total of 614,400 potential combinations

PARAM_GRID = {
    'n_estimators': [500, 1000, 1500, 2000, 2500, 3000],
    'learning_rate': [0.005, 0.01, 0.02, 0.03, 0.05],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [3, 5, 7, 10],
    'subsample': [0.7, 0.8, 0.85, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.85, 0.9],
    'reg_lambda': [0.5, 1.0, 1.5, 2.0, 3.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0],
    'gamma': [0, 0.1, 0.3, 0.5],
}

# Prints total number of combos
total_combos = np.prod(
    [len(v) for v in PARAM_GRID.values()]
)

print(f"Total possible combos: {total_combos:,}")

In [ ]:
# Helper functions

# drop dups, preserves order
def unique_list(seq):
    return list(dict.fromkeys(seq))

# drop dup col names
def dedupe_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[:, ~df.columns.duplicated()].copy()

def is_pitcher(pos) -> int:
    PITCHER_PREFIXES = ("P", "SP", "RP", "RHP", "LHP")
    if pd.isna(pos):
        return 0
    s = str(pos).strip().upper()
    # handles pitcher positional variations
    return int(s.startswith(PITCHER_PREFIXES) or ("RHP" in s) or ("LHP" in s))

def time_split(df: pd.DataFrame, year_col: str = "year") -> Tuple[pd.DataFrame, pd.DataFrame]:
    y = pd.to_numeric(df[year_col], errors="coerce").astype("Int64")
    train = df[y.isin(TRAIN_YEARS)].copy()
    test = df[y.isin(TEST_YEARS)].copy()
    return train, test

# Data integrity check
def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    if mask.sum() == 0:
        raise ValueError("No finite y_true/y_pred pairs available.")
    
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae = float(mean_absolute_error(y_true, y_pred))
    r2 = float(r2_score(y_true, y_pred))
    
    return {"RMSE": rmse, "MAE": mae, "R2": r2}

def safe_cols(df: pd.DataFrame, col_list: List[str]) -> List[str]:
    return [c for c in col_list if c in df.columns]

In [ ]:
# Feature Engineering
# replcaited from v1

def _safe_numeric(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def _weighted_mean(series: pd.Series, weights: pd.Series) -> float:
    s = pd.to_numeric(series, errors="coerce")
    w = pd.to_numeric(weights, errors="coerce").fillna(0.0)
    mask = np.isfinite(s) & np.isfinite(w) & (w > 0)
    if mask.sum() == 0:
        s2 = s[np.isfinite(s)]
        return float(s2.mean()) if len(s2) else np.nan
    return float(np.average(s[mask], weights=w[mask]))

def add_pre_rate_features(
    contracts: pd.DataFrame,
    season_rates: pd.DataFrame,
    *,
    rate_cols: List[str],
    weight_col: str | None,
    prefix: str,
    pre_years: int = 3,
) -> pd.DataFrame:
    """
    Pre-window aggregate computations for seasons in window
    """    
    dfc = contracts.copy()
    dfc["key_fangraphs"] = pd.to_numeric(dfc["key_fangraphs"], errors="coerce")
    dfc["year"] = pd.to_numeric(dfc["year"], errors="coerce")
    dfc["_row_id"] = np.arange(len(dfc), dtype=int)

    dfs = season_rates.copy()
    dfs["playerId"] = pd.to_numeric(dfs["playerId"], errors="coerce")
    dfs["Season"] = pd.to_numeric(dfs["Season"], errors="coerce")
    dfs = _safe_numeric(dfs, rate_cols + ([weight_col] if weight_col else []))

    m = dfc[["_row_id", "key_fangraphs", "year"]].merge(
        dfs, left_on="key_fangraphs", right_on="playerId", how="left"
    )

    m["lb_start"] = m["year"] - pre_years
    m["lb_end"] = m["year"] - 1
    m = m[m["Season"].between(m["lb_start"], m["lb_end"], inclusive="both")].copy()

    out = dfc.copy()

    cov = m.groupby("_row_id")["Season"].nunique().rename(f"{prefix}_pre_seasons")
    out = out.merge(cov, left_on="_row_id", right_index=True, how="left")
    out[f"{prefix}_pre_seasons"] = out[f"{prefix}_pre_seasons"].fillna(0).astype(int)

    if weight_col and weight_col in m.columns:
        rel_sum = m.groupby("_row_id")[weight_col].sum(min_count=1).rename(f"{prefix}_pre_reliability_sum")
        out = out.merge(rel_sum, left_on="_row_id", right_index=True, how="left")
        out[f"{prefix}_pre_reliability_sum"] = pd.to_numeric(out[f"{prefix}_pre_reliability_sum"], errors="coerce").fillna(0.0)

    for rc in rate_cols:
        feat_name = f"{prefix}_pre_{rc}"
        if rc not in m.columns:
            out[feat_name] = np.nan
            continue

        if weight_col and weight_col in m.columns:
            agg = m.groupby("_row_id").apply(lambda g: _weighted_mean(g[rc], g[weight_col])).rename(feat_name)
        else:
            agg = m.groupby("_row_id")[rc].mean().rename(feat_name)

        out = out.merge(agg, left_on="_row_id", right_index=True, how="left")

    out[f"has_{prefix}_pre"] = (out[f"{prefix}_pre_seasons"] > 0).astype(int)
    out = out.drop(columns=["_row_id"])
    return out

In [ ]:
# apply pre panel features

def add_pre_panel_features(
    contracts: pd.DataFrame,
    panel: pd.DataFrame,
    *,
    contract_key_col: str,
    panel_key_col: str,
    contract_year_col: str,
    panel_year_col: str,
    feature_cols: List[str],
    weight_col: str | None,
    prefix: str,
    pre_years: int = 3,
) -> pd.DataFrame:
    """ Generic season panel aggregator (only used for defense and statcast files) """
    # reduced implementation of data integrity checks since data
    # is known to be clean due to success of the baseline model

    # temp cols
    out = contracts.copy()
    out["_row_id"] = np.arange(len(out), dtype=int)

    dfc = out[["_row_id", contract_key_col, contract_year_col]].copy()
    dfc["_contract_key"] = pd.to_numeric(dfc[contract_key_col], errors="coerce")
    dfc["_contract_year"] = pd.to_numeric(dfc[contract_year_col], errors="coerce")

    keep_feats = [c for c in feature_cols if c in panel.columns]
    keep_cols = [panel_key_col, panel_year_col] + keep_feats
    if weight_col and weight_col in panel.columns and weight_col not in keep_cols:
        keep_cols.append(weight_col)

    p = panel[keep_cols].copy()
    p["_panel_key"] = pd.to_numeric(p[panel_key_col], errors="coerce")
    p["_panel_year"] = pd.to_numeric(p[panel_year_col], errors="coerce")

    for c in keep_feats:
        p[c] = pd.to_numeric(p[c], errors="coerce")
    if weight_col and weight_col in p.columns:
        p[weight_col] = pd.to_numeric(p[weight_col], errors="coerce")

    merge_cols = ["_panel_key", "_panel_year"] + keep_feats + ([weight_col] if (weight_col and weight_col in p.columns) else [])
    m = dfc[["_row_id", "_contract_key", "_contract_year"]].merge(
        p[merge_cols], left_on="_contract_key", right_on="_panel_key", how="left"
    )

    # Apply lookback filter
    m["lb_start"] = m["_contract_year"] - pre_years
    m["lb_end"] = m["_contract_year"] - 1
    m = m[m["_panel_year"].between(m["lb_start"], m["lb_end"], inclusive="both")].copy()

    cov = m.groupby("_row_id")["_panel_year"].nunique().rename(f"{prefix}_pre_seasons")
    out = out.merge(cov, on="_row_id", how="left")
    out[f"{prefix}_pre_seasons"] = out[f"{prefix}_pre_seasons"].fillna(0).astype(int)

    # Weighted sums
    if weight_col and weight_col in m.columns:
        wsum = m.groupby("_row_id")[weight_col].sum(min_count=1).rename(f"{prefix}_pre_weight_sum")
        out = out.merge(wsum, on="_row_id", how="left")
        out[f"{prefix}_pre_weight_sum"] = pd.to_numeric(out[f"{prefix}_pre_weight_sum"], errors="coerce").fillna(0.0)

    # Aggregate features where necessary
    for fc in keep_feats:
        feat_name = f"{prefix}_pre_{fc}"
        if fc not in m.columns:
            out[feat_name] = np.nan
            continue

        if weight_col and weight_col in m.columns:
            agg = m.groupby("_row_id").apply(lambda g: _weighted_mean(g[fc], g[weight_col])).rename(feat_name)
        else:
            agg = m.groupby("_row_id")[fc].mean().rename(feat_name)

        out = out.merge(agg, on="_row_id", how="left")

    out[f"has_{prefix}_pre"] = (out[f"{prefix}_pre_seasons"] > 0).astype(int)
    out = out.drop(columns=["_row_id"])
    return out

In [ ]:
# Feature Engineering for v2

def engineer_v2_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    v2 feature engineering
    Emphasis on feature interactions and injury-specific integrations
    """
    df = df.copy()

    # tells model to use looback=5 and lambda=35
    isi_col = 'ISI_lb5_lamdba_35'
    
    # Data Integrity
    if isi_col not in df.columns:
        print(f"WARNING: {isi_col} not found.")
        return df
    
    feature_count_start = len(df.columns)

##############
# New Features
##############

    # Age interactions
    # attempting to show that older players + injuries = larger injury discount
    # age at signing col NECESSARY
    if 'age_at_signing' in df.columns:
        df['isi_x_age'] = df[isi_col] * df['age_at_signing']
        # applies a squared function to age
        # exponential age factor
        df['isi_x_age_squared'] = df[isi_col] * (df['age_at_signing'] ** 2)
        # 'old' players defined as 32 years old or higher
        df['isi_x_old'] = df[isi_col] * (df['age_at_signing'] >= 32).astype(int)
    
    # contract term length interactions
    if 'years_int' in df.columns:
        df['isi_x_years'] = df[isi_col] * df['years_int']
        # for analysis, long term is 4-5 years (6+ term years are excluded)
        df['isi_x_longterm'] = df[isi_col] * (df['years_int'] >= 4).astype(int)
    
    # advanced batting metric interactions
    for stat in safe_cols(df, ['bat_pre_wRC+', 'bat_pre_OPS', 'bat_pre_wOBA']):
        df[f'isi_x_{stat}'] = df[isi_col] * df[stat]
    
    # advanced pitching metric interactions
    for stat in safe_cols(df, ['pit_pre_FIP', 'pit_pre_strikeout_percent']):
        df[f'isi_x_{stat}'] = df[isi_col] * df[stat]
    
    # prints the total number of interaction features that were created
    print(f"Total interaction features: {len([c for c in df.columns if 'isi_x_' in c])}")
    

    # injury type decomposition
    # utilizes two lookback varations, one lambda
    for isi_variant in ['ISI_lb3_lamdba_35', 'ISI_lb5_lamdba_35']:
        if isi_variant not in df.columns:
            continue
        
        suffix = isi_variant.replace('ISI', '')
        
        # Surgery factor
        surgery_col = f'any_surgery_flag{suffix}'
        if surgery_col in df.columns:
            df[f'surgery_burden{suffix}'] = df[isi_variant] * df[surgery_col]
            df[f'no_surgery_but_isi{suffix}'] = df[isi_variant] * (1 - df[surgery_col])
        
        # Structural damage factor
        structural_col = f'any_structural_flag{suffix}'
        if structural_col in df.columns:
            df[f'structural_burden{suffix}'] = df[isi_variant] * df[structural_col]
        
        # Arm injury factor
        # pitcher identiciation only
        arm_share_col = f'arm_share_days{suffix}'
        if arm_share_col in df.columns and 'is_pitcher_flag' in df.columns:
            df[f'pitcher_arm_burden{suffix}'] = df[arm_share_col] * df['is_pitcher_flag']
            df[f'pitcher_isi_x_arm{suffix}'] = df[isi_variant] * df[arm_share_col] * df['is_pitcher_flag']
    
    # prints the total number of interaction features that were created
    print(f"Created {len([c for c in df.columns if 'burden' in c])} injury type features")

    # positional factor
    df['isi_pitcher'] = df[isi_col] * df['is_pitcher_flag']
    df['isi_position_player'] = df[isi_col] * (1 - df['is_pitcher_flag'])
    
    # defines starter and reliver positions where available
    if 'position' in df.columns:
        df['isi_starter'] = df[isi_col] * (df['position'] == 'SP').astype(int)
        df['isi_reliever'] = df[isi_col] * (df['position'] == 'RP').astype(int)
        
        # premium positions defined as positions where scarcity exists
        # ADJUST AS NEEDED
        premium_pos = ['SS', 'C', 'CF']
        df['isi_premium_pos'] = df[isi_col] * df['position'].isin(premium_pos).astype(int)
    
    # prints the total number of interaction features that were created
    print(f"Created {len([c for c in df.columns if c.startswith('isi_') and any(x in c for x in ['pitcher', 'position', 'starter', 'reliever', 'premium'])])} position features")
    
    
    # temporal features
    # utilizes both a three and five year lookback period
    isi_recent = 'ISI_lb3_lamdba_35'
    isi_full = 'ISI_lb5_lamdba_35'
    
    if isi_recent in df.columns and isi_full in df.columns:
        # ISI acceleration feature intended to diagnose health changes
        # improving is good, worsening signals a durability decrease
        df['isi_acceleration'] = df[isi_recent] - df[isi_full]
        df['isi_worsening'] = (df['isi_acceleration'] > 0).astype(int)
        df['isi_improving'] = (df['isi_acceleration'] < -0.05).astype(int)
        
        df['isi_trend_ratio'] = df[isi_recent] / (df[isi_full] + 0.01)
            
    # Fast vs slow decay (recency signal)
    # utilizes a fast and slow recency weight
    # lookback=5 lambda=0.70 (increased emphasis on more recent injuries)
    # lookback=5 lambda=0.35 (injury history is more uniform)
    isi_fast = 'ISI_lb5_lamdba_7'
    isi_slow = 'ISI_lb5_lamdba_35'
    
    # defines signals based on decay speeds
    if isi_fast in df.columns and isi_slow in df.columns:
        df['isi_recency_signal'] = df[isi_fast] - df[isi_slow]
        df['injuries_very_recent'] = (df['isi_recency_signal'] > 0.1).astype(int)
    
    # non-linear transformation
    # attempting to see if applying transformations
    # to the ISI cols can impact results

    # polynomial transformations
    # Is there an exponential ISI discount?
    df[f'{isi_col}_squared'] = df[isi_col] ** 2
    df[f'{isi_col}_sqrt'] = np.sqrt(df[isi_col])
    df[f'{isi_col}_log1p'] = np.log1p(df[isi_col])
    
    # defines thresholds
    isi_75 = df[isi_col].quantile(0.75)
    isi_25 = df[isi_col].quantile(0.25)
    
    df['isi_high_burden'] = (df[isi_col] > isi_75).astype(int)
    df['isi_clean_history'] = (df[isi_col] <= isi_25).astype(int)

    feature_count_end = len(df.columns)
    new_features = feature_count_end - feature_count_start
    
    # page split
    print("\n" + "="*20)
    print(f"New features created: {new_features}")
    print(f"Total cols: {feature_count_end}")
    print("="*20)
    
    return df

In [ ]:
# Data Preprocessing

def make_preprocessor(numeric_features: List[str], categorical_features: List[str]) -> ColumnTransformer:
    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    
    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])
    
    return ColumnTransformer(
        transformers=[
            ("num", num_pipe, numeric_features),
            ("cat", cat_pipe, categorical_features),
        ],
        remainder="drop"
    )


In [ ]:
# Brute force CV Grid Search
# replicates v1 strucutre

def brute_force_search(
    X_train, y_train, X_test, y_test,
    numeric_features, categorical_features,
    variant_name, time_budget_seconds
):
    """Brute force hypterparameters until maximum time runs out"""
    # page split
    print(f"\n{'='*80}")
    print(f"VARIANT: {variant_name}")
    print(f"{'='*80}")
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")
    print(f"Features: {len(numeric_features)} numeric + {len(categorical_features)} categorical")
    print(f"Time budget: {time_budget_seconds/60:.1f} minutes")
    
    # runs proprocessor function created above
    preprocessor = make_preprocessor(numeric_features, categorical_features)

    # generates param combos
    param_keys = list(PARAM_GRID.keys())
    param_values = [PARAM_GRID[k] for k in param_keys]
    all_combinations = list(product(*param_values))

    # shuffle param combos to allow for a randomized grid search
    np.random.seed(RANDOM_STATE)
    np.random.shuffle(all_combinations)

    # Results tracker
    results = []
    start_time = time.time()
    best_mae = float('inf')
    best_params = None
    
    # TimeSeriesSplit for CV (refer to documentation for this)
    # CV splits limited to three since the dataset is small (n < 1000 rows)
    tscv = TimeSeriesSplit(n_splits=3)
    
    for i, combo in enumerate(all_combinations):
        # Check time limit
        # prints number of combos tested after the time limit expires
        elapsed = time.time() - start_time
        if elapsed >= time_budget_seconds:
            print(f"\n[TIME LIMIT] Stopped after {elapsed/60:.1f} minutes, tested {i} combinations")
            break

        # create param dict to store results        
        params = dict(zip(param_keys, combo))
        
        try:
            base_model = XGBRegressor(
                **params,
                random_state=RANDOM_STATE,
                objective="reg:squarederror",
                n_jobs=1,
                verbosity=0
            )
            
            pipeline = Pipeline([
                ("pre", preprocessor),
                ("model", base_model)
            ])
            
            # log transformation
            ttr = TransformedTargetRegressor(
                regressor=pipeline,
                func=np.log1p,
                inverse_func=np.expm1
            )
            
            # CV eval
            cv_maes = []
            for train_idx, val_idx in tscv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                
                ttr.fit(X_tr, y_tr)
                y_pred = ttr.predict(X_val)
                mae = mean_absolute_error(y_val, y_pred)
                cv_maes.append(mae)
            
            cv_mae = np.mean(cv_maes)
            
            # test set eval
            ttr.fit(X_train, y_train)
            y_test_pred = ttr.predict(X_test)
            test_metrics = regression_metrics(y_test, y_test_pred)

            # records best results
            # prints an update statement on improved results
            if test_metrics['MAE'] < best_mae:
                best_mae = test_metrics['MAE']
                best_params = params
                print(f"\n[NEW BEST] Iteration {i+1}")
                print(f"  CV MAE: ${cv_mae:,.0f}")
                print(f"  Test MAE: ${test_metrics['MAE']:,.0f}")
                print(f"  Test R²: {test_metrics['R2']:.4f}")

            # append results
            results.append({
                'cv_mae': cv_mae,
                'test_mae': test_metrics['MAE'],
                'test_rmse': test_metrics['RMSE'],
                'test_r2': test_metrics['R2'],
                **params
            })

            # Displays update every 10 iters
            if (i + 1) % 10 == 0:
                elapsed = time.time() - start_time
                rate = (i + 1) / elapsed
                remaining = time_budget_seconds - elapsed
                est_remaining_iters = int(rate * remaining)
                print(f"  [{i+1:4d}] Elapsed: {elapsed/60:.1f}m | Rate: {rate:.1f} iter/s | Est. remaining: {est_remaining_iters}")

        # prevents system crash if a model fails
        except Exception as e:
            print(f"  [ERROR] Iteration {i+1}: {e}")
            continue
    
    elapsed_total = time.time() - start_time
    # page split
    print(f"\n{'='*80}")
    print(f"COMPLETED {variant_name}")
    print(f"Tested {len(results)} combinations in {elapsed_total/60:.1f} minutes")
    print(f"Best Test MAE: ${best_mae:,.0f}")
    print("="*80)
    
    return {
        "variant": variant_name,
        "best_params": best_params,
        "best_test_mae": best_mae,
        "n_tested": len(results),
        "elapsed_seconds": elapsed_total,
        "all_results": results
    }


In [ ]:
# Brute force main run

if __name__ == "__main__":
    
    # page split
    print("="*20)
    print("Ruunning Regression V2")
    print("="*20)
    
    # Data loader
    df = pd.read_csv(DATA_PATH, low_memory=False)
    bat_rates = pd.read_csv(BAT_RATES_PATH, low_memory=False)
    pit_rates = pd.read_csv(PIT_RATES_PATH, low_memory=False)
    def_stats = pd.read_csv(DEF_STATS_PATH, low_memory=False)
    sc_pit = pd.read_csv(STATCAST_PIT_PATH, low_memory=False)
    
    # Filter contracts
    df = df[pd.to_numeric(df["years_int"], errors="coerce") <= MAX_YEARS].copy()
    df["year"] = pd.to_numeric(df["term_start_year"], errors="coerce").astype("Int64")
    df["is_pitcher_flag"] = df["position"].apply(is_pitcher)
    
    # displays number of contracts post filtering
    print(f"Contracts after filtering: {len(df)}")

    # defines baseline features and coverage reports
    BAT_EXCLUDE = {"playerId", "Season", "Name", "Tm", "PA", "bat_rate_reliability"}
    bat_rate_cols = [c for c in bat_rates.columns if c not in BAT_EXCLUDE and not c.endswith("_dup")]
    bat_weight_col = "bat_rate_reliability" if "bat_rate_reliability" in bat_rates.columns else None
    
    df = add_pre_rate_features(df, bat_rates, rate_cols=bat_rate_cols, weight_col=bat_weight_col, prefix="bat", pre_years=3)
    print(f"  Batting: {df['has_bat_pre'].mean():.1%} coverage")
    
    PIT_EXCLUDE = {"playerId", "Season", "Name", "Tm", "IP", "TBF", "pit_rate_reliability"}
    pit_rate_cols = [c for c in pit_rates.columns if c not in PIT_EXCLUDE and not c.endswith("_dup")]
    pit_weight_col = "pit_rate_reliability" if "pit_rate_reliability" in pit_rates.columns else None
    
    df = add_pre_rate_features(df, pit_rates, rate_cols=pit_rate_cols, weight_col=pit_weight_col, prefix="pit", pre_years=3)
    print(f"  Pitching: {df['has_pit_pre'].mean():.1%} coverage")
    
    def_feature_cols = safe_cols(def_stats, ["defensive_runs_saved", "fielding_percentage", "Errors"])
    df = add_pre_panel_features(
        df, def_stats,
        contract_key_col="key_mlbam", panel_key_col="MLBAMID",
        contract_year_col="year", panel_year_col="year",
        feature_cols=def_feature_cols,
        weight_col="Innings_played" if "Innings_played" in def_stats.columns else None,
        prefix="def", pre_years=3
    )
    print(f"  Defense: {df['has_def_pre'].mean():.1%} coverage")
    
    sc_feature_cols = safe_cols(sc_pit, ["fastball_avg_speed", "whiff_percent", "hard_hit_percent", 
                                          "barrel_batted_rate", "exit_velocity_avg", "swing_percent"])

# apply features to the dataset
    df = add_pre_panel_features(
        df, sc_pit,
        contract_key_col="key_mlbam", panel_key_col="player_id",
        contract_year_col="year", panel_year_col="year",
        feature_cols=sc_feature_cols,
        weight_col="pa" if "pa" in sc_pit.columns else None,
        prefix="scpit", pre_years=3
    )
    print(f"  Statcast: {df['has_scpit_pre'].mean():.1%} coverage")
    
    # Data integrity
    df = dedupe_columns(df)
    
    # Apply newly created features
    df = engineer_v2_features(df)
    
    # regression setup
    df_reg = df[pd.notna(df[REG_TARGET])].copy()
    df_reg[REG_TARGET] = pd.to_numeric(df_reg[REG_TARGET], errors="coerce")
    df_reg = df_reg[np.isfinite(df_reg[REG_TARGET]) & (df_reg[REG_TARGET] >= 0)].copy()
    
    train_reg, test_reg = time_split(df_reg, year_col="year")
    
    # apply AAV autoff
    aav_cutoff = train_reg[REG_TARGET].quantile(REMOVE_TOP_PCTL)
    train_reg_cut = train_reg[train_reg[REG_TARGET] <= aav_cutoff].copy()
    test_reg_cut = test_reg[test_reg[REG_TARGET] <= aav_cutoff].copy()
    
    # displays size of training and testing datasets (post AAV cutoff)
    print(f"[TOP5%] Cutoff: {aav_cutoff:,.0f}")
    print(f"[TOP5%] Train: {len(train_reg_cut)} | Test: {len(test_reg_cut)}")
    
    # feature lists
    BASE_NUMERIC = ["age_at_signing", "years_int", "opt_out_flag", "year", "is_pitcher_flag"]
    BASE_CATEGORICAL = ["position", "qualifying_offer"]
    
    cov_suffixes = ("_pre_seasons", "_pre_reliability_sum", "_pre_weight_sum")
    generated_cov_feats = [
        c for c in df_reg.columns
        if c in {"has_bat_pre", "has_pit_pre", "has_def_pre", "has_scpit_pre"}
        or c.endswith(cov_suffixes)
    ]
    
    PREFIXES = ("bat_pre_", "pit_pre_", "def_pre_", "scpit_pre_")
    generated_rate_feats = [
        c for c in df_reg.columns
        if c.startswith(PREFIXES) and c not in generated_cov_feats
    ]
    
    base_numeric_features = unique_list([
        c for c in (BASE_NUMERIC + generated_rate_feats + generated_cov_feats)
        if c in df_reg.columns
    ])
    
    categorical_features = unique_list([
        c for c in BASE_CATEGORICAL
        if c in df_reg.columns
    ])
    
    print(f"\nBaseline features: {len(base_numeric_features)} numeric + {len(categorical_features)} categorical")
    
    # Test variants
    variants_to_test = [
        {
            'name': 'BASELINE_V2_RERUN',
            'numeric': base_numeric_features,
            'categorical': categorical_features,
            'description': 'Baseline rerun to verify v1 results'
        },
        {
            'name': 'V2_ALL_FEATURES',
            'numeric': unique_list(base_numeric_features + [c for c in df_reg.columns if 'isi' in c.lower() and c not in base_numeric_features]),
            'categorical': categorical_features,
            'description': 'All v2 engineered features'
        },
        {
            'name': 'V2_INTERACTIONS_ONLY',
            'numeric': unique_list(base_numeric_features + [c for c in df_reg.columns if 'isi_x_' in c]),
            'categorical': categorical_features,
            'description': 'ISI interactions only'
        },
        {
            'name': 'V2_INJURY_TYPES',
            'numeric': unique_list(base_numeric_features + [c for c in df_reg.columns if 'burden' in c or c.startswith('had_')]),
            'categorical': categorical_features,
            'description': 'Injury type decomposition'
        },
    ]

    for v in variants_to_test:
        print(f"{v['name']:25s}: {len(v['numeric'])} numeric features")
        print(f"{'':25s}  {v['description']}")
    
    # time limit rules
    total_seconds = TIME_LIMIT_HOURS * 3600
    time_per_variant = total_seconds / len(variants_to_test)
    
    all_results = []
    overall_start = time.time()
    
    for variant in variants_to_test:
        elapsed_total = time.time() - overall_start
        if elapsed_total >= total_seconds:
            print(f"\n[GLOBAL TIME LIMIT] Stopping")
            break
        
        # filter for available features
        available_numeric = [f for f in variant['numeric'] if f in train_reg_cut.columns and f in test_reg_cut.columns]
        available_categorical = [f for f in variant['categorical'] if f in train_reg_cut.columns and f in test_reg_cut.columns]
        
        all_features = available_numeric + available_categorical
        
        try:
            result = brute_force_search(
                train_reg_cut[all_features],
                train_reg_cut[REG_TARGET],
                test_reg_cut[all_features],
                test_reg_cut[REG_TARGET],
                available_numeric,
                available_categorical,
                variant['name'],
                time_per_variant
            )
            all_results.append(result)
        except Exception as e:
            print(f"ERROR in {variant['name']}: {e}")
            import traceback
            traceback.print_exc()
    
    # page split
    # Save results
    print("\n" + "="*20)
    print("SAVING RESULTS")
    print("="*20)

    # Summary results
    if all_results:
        summary = pd.DataFrame([
            {
                "variant": r["variant"],
                "best_test_mae": r["best_test_mae"],
                "n_combinations_tested": r["n_tested"],
                "elapsed_minutes": r["elapsed_seconds"] / 60,
                "best_params": str(r["best_params"])
            }
            for r in all_results
        ])
        
        summary = summary.sort_values("best_test_mae")
        summary.to_csv(OUT_RESULTS, index=False)
        
        print(f"\nSaved summary to: {OUT_RESULTS}")
        # page split
        print("\n" + "="*80)
        print("FINAL RESULTS")
        print("="*80)
        print(summary[["variant", "best_test_mae", "n_combinations_tested"]].to_string(index=False))
        
        # saves detailed results for each varaint tested
        for r in all_results:
            variant_file = OUT_RESULTS.replace(".csv", f"_{r['variant']}_detailed.csv")
            detailed_df = pd.DataFrame(r["all_results"])
            detailed_df = detailed_df.sort_values("test_mae")
            detailed_df.to_csv(variant_file, index=False)
            print(f"\nSaved: {variant_file}")

MLB CONTRACT REGRESSION V2

Loading data...
Contracts after filtering: 844

Building baseline features...
  Batting: 33.2% coverage
  Pitching: 39.6% coverage
  Defense: 16.6% coverage
  Statcast: 16.8% coverage

V2 FEATURE ENGINEERING

1. Creating ISI × Performance/Age interactions...
   Created contract length interactions
   Total interaction features: 7

2. Decomposing injury types (surgery, structural, arm)...
   Created 12 injury type features

3. Creating position-specific ISI encoding...
   Created 5 position features

4. Creating ISI temporal trend features...
   Created ISI temporal trend features

5. Creating nonlinear ISI transformations...
   Created 5 nonlinear features

V2 FEATURE ENGINEERING COMPLETE
  New features created: 33
  Total columns: 214

Train: 381 | Test: 191
Top 5% cutoff: $22,361,449

Baseline features: 79 numeric + 1 categorical

VARIANTS TO TEST
BASELINE_V2_RERUN        : 79 numeric features
                           Baseline rerun to verify v1 results
